> ChatGPT *de juguete*

Vamos construir un *ChatGPT* "mini" (lo suficientemente pequeño como para entenderlo en unos minutos, pero lo suficientemente real como para ver la *magia* de la predicción de texto). En este notebook seguiremos el enfoque de [Andrej Karpathy](https://karpathy.ai/): escribir el código desde cero, hacerlo de manera muy simple, y entrenar un pequeño modelo que pueda aprender a generar texto carácter a carácter. No estamos buscando potencia o velocidad. El objetivo es desmitificar cómo funcionan realmente estos modelos por dentro. Al final, tendrás una comprensión práctica de cómo se puede construir un sistema tipo GPT paso a paso, y podrás jugar con él y experimentar.

Este *notebook* está basado en los repositorios [ng-video-lecture](https://github.com/karpathy/ng-video-lecture/tree/master) y [minGPT](https://github.com/karpathy/minGPT) de Karpathy, y el [video tutorial complementario](https://www.youtube.com/watch?v=kCc8FmEb1nY).

# Preparación del entorno

En principio, podrías ejecutar el *notebook* en *Colab* o localmente. ¿El *notebook* se está ejecutando en *Colab*?

In [ ]:
try:
    import google.colab
    running_in_colab = True
except ImportError:
    running_in_colab = False

running_in_colab

## GPU

Para ejecutar este *notebook* (en un tiempo razonable), utilizaremos la [unidad de procesamiento gráfico](https://en.wikipedia.org/wiki/Graphics_processing_unit) (GPU) que proporciona el entorno *Colab*. Para habilitarla, en la parte superior derecha de la interfaz de *Colab*, haz clic en `Conectar`, `Cambiar tipo de entorno de ejecución`, selecciona `GPU T4` y, a continuación, haz clic en `Guardar`. Las llamadas `to_device` que aparecen dispersas por todo el *notebook* tienen por objeto *mover* las matrices a la GPU (si hay una disponible).

Si no se ejecuta en *Colab*, es posible que quieras elegir una GPU si hay varias disponibles. Ignora esto si se ejecuta en *Colab*.

In [ ]:
if not running_in_colab:

    import os
    os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

## Bibliotecas de Python

Algunos `import`s necesarios se "centralizan" aquí.

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F

Hay una GPU disponible?

In [ ]:
torch.cuda.is_available()

# Preparación de datos

Vamos a descargar un texto. El código de abajo descargará los textos de Shakespeare, pero puedes usar esencialmente cualquier recurso de texto (un libro, alguna página web...) que te guste!!

In [ ]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

Se lee en memoria el archivo *completo*.

In [ ]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()
print('number of characters read: ', len(text))

Vamos a echar un vistazo a nuestro conjunto de datos.

<font color='red'>TO-DO</font>: Muestra los primeros 200 caracteres.

## Vocabulario

Como es habitual, para convertir texto en números necesitamos un **vocabulario** que nos permita construir, *pieza* a *pieza*, todo el dataset. Por simplicidad, vamos a considerar los caracteres individuales que aparecen en el texto.

In [ ]:
chars = sorted(set(text))
vocab_size = len(chars)
print(rf'Vocabulary ({len(chars)} elements) is: {''.join(chars)} ')

Vamos a asociar un índice (`i`) a cada elemento (carácter, `s`) del vocabulario. Cualquier *mapeo* nos sirve, así que lo más fácil es asociar cada carácter con su índice en la lista.

In [ ]:
stoi = { ch:i for i, ch in enumerate(chars) }

(`stoi` del inglés *string to integer*)

Puedes utilizarlo para averiguar el índice de cualquier carácter que quieras, p. ej.,

<font color='red'>TO-DO</font>: ¿Cuál es el índice del carácter `k`?

También necesitamos el *mapeo* inverso, es decir, de índice a carácter

In [ ]:
itos = chars

<font color='red'>TO-DO</font>: ¿Cuál es el carácter asociado con el índice `12`?

Aprovechamos los *mapeos* anteriores (un `dict` y una `list`, respectivamente) para hacer funciones capaces de operar, respectivamente, sobre **secuencias** de caracteres (para la *codificación*)...

In [ ]:
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
print(encode("hii there"))

...y números (para la *decodificación*)

In [ ]:
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string
print(decode(encode("hii there")))

Vamos a codificar (caracteres a números) el conjunto de datos en un `Tensor` de *PyTorch*

In [ ]:
data = torch.tensor(encode(text), dtype=torch.long)

Tiene el mismo número de elementos que `text` arriba, y cada uno de los elementos es un entero de 64 bits (`torch.int64`).

In [ ]:
assert len(text) == len(data)
print(data.dtype)

Imprimimos los primeros caracteres, ahora representados como números

In [ ]:
print(data[:200])

## Partición training/validation

Los datos se dividen en conjuntos de *training* y *validation*, de modo que tengamos una forma de saber como de bien está generalizando el modelo.

In [ ]:
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

## Tamaño de bloque

Como no podemos procesar *todos* los datos de una vez (a menos que tengas un dataset muy pequeño, lo cual probablemente acabaría dando lugar a un modelo muy malo), necesitamos *fragmentarlos*. Consideremos fragmentos de tamaño

In [ ]:
block_size = 8

[GPT-4](https://en.wikipedia.org/wiki/GPT-4), por ejemplo, tiene un tamaño de bloque (también conocido como *context length*) de decenas de miles de *tokens*, cada uno de ellos abarcando más de un carácter. Por tanto, ten en cuenta que estamos, por supuesto, mirando un ejemplo de juguete

Echemos un vistazo al primer bloque

In [ ]:
train_data[:block_size]

Al procesar cada bloque, el objetivo es predecir un carácter dados *todos* los anteriores: para predecir el 2º carácter solo usaremos el 1º, al predecir el 3º carácter, usaremos el 1º y 2º...y así sucesivamente. En principio, esto significaría que, para cada tamaño de bloque, estaríamos haciendo `block_size - 1` predicciones. Siempre se considera un carácter extra, el carácter `(block_size+1)`-ésimo, de modo que tengamos exactamente `block_size` predicciones. Por tanto, el bloque anterior producirá las siguientes tareas de predicción:

In [ ]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f'When input is {context.tolist()}, the target: {target}')

En última instancia, esto es un problema de clasificación *multiclase*: cada predicción no es solo una etiqueta, sino una **distribución de probabilidad completa** sobre todas las clases posibles. En nuestro caso, refleja qué probabilidad tiene cada carácter del vocabulario de ser el siguiente.

## Batching
Como queremos aprovechar al máximo las GPUs (procesamiento paralelo), *empaquetaremos* y procesaremos varios bloques al mismo tiempo...tantos como

In [ ]:
batch_size = 4

Vamos a fijar la semilla del generador de números pseudo-aleatorios (PRNG) para que obtengamos siempre los mismos resultados (al generalos números "aleatorios" más abajo).

In [ ]:
torch.manual_seed(1337)

Una función auxiliar para ensamblar un batch aleatorio, ya sea del conjunto de *training* o del de *validation* (dependiendo del valor del parámetro `split`).

In [ ]:
def get_batch(split: str):
    
    # either the training or validation set
    data = train_data if split == 'train' else val_data

    # a random index (for each block in the batch) that is followed by at least `block_size` characters so that we can extract a full block
    ix = torch.randint(len(data) - block_size, (batch_size,))
    
    # notice the `stack`ing
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    
    return x, y

Vamos a generar un batch del conjunto de *training*

In [ ]:
xb, yb = get_batch('train')

...y echemos un vistazo a las entradas y salidas.

- **Input** (entrada) y sus dimensiones (`bath_size`, `block_size`)

In [ ]:
print(xb)
print(xb.shape)

- **Target** (salida) y sus dimensiones (`bath_size`, `block_size`)

In [ ]:
print(yb)
print(yb.shape)

<font color='red'>TO-DO</font>: muestra el *texto* representado por `yb` (las salidas o targets)

Observa que

In [ ]:
xb.shape == yb.shape

El batch anterior plantea los siguientes problemas de predicción:

In [ ]:
# for every sequence in the batch...
for i_b, b in enumerate(range(batch_size)):

    print(f'{i_b}-th element in the batch:')
    
    # for every element in the sequence...
    for t in range(block_size):
        
        # every character in the sequence up to and including (hence the `+1`) t
        context = xb[b, :t+1]
        
        # by construction (above), `yb[b,t]` is the target for the sequence up to and including t
        target = yb[b,t]
        
        print(f"When input is {context.tolist()}, the target: {target}")

    print('-'*5)

Lo que debe entrar en la red neuronal (NN) son en realidad `tensor`es y **no** `list`s de tamaño *variable*. La entrada a la NN será

In [ ]:
xb

y la salida correspondiente (*target*).

In [ ]:
yb

Esto da lugar a `bath_size` $\times$ `block_size` (las dimensiones de `xb` e `yb`) predicciones *independientes* para que el modelo aprenda ([explicación de Karpathy](https://youtu.be/kCc8FmEb1nY?t=1281)). Todas se procesan simultáneamente.

# Entrenamiento

## Parámetros

Configuramos algunos (hiper)parámetros que utilizamos durante el entrenamiento del modelo

- Vistos arriba

In [ ]:
block_size = 32
batch_size = 16

- ¿Cuántos batches (aleatorios) para entrenar el modelo?

In [ ]:
# max_iters = 5000
max_iters = 500

- Algunos parámetros específicos de la arquitectura

In [ ]:
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0

* En relación con el entrenamiento

In [ ]:
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200

## Modelo

Esta es la definición de la NN (basada en [transformers](https://es.wikipedia.org/wiki/Transformador_(modelo_de_aprendizaje_autom%C3%A1tico))). **Ignórala** por ahora: en este curso todavía no nos interesan los detalles de implementación, aunque, como puedes ver, el código no es realmente grande. Si profundizas en el código (de nuevo, no es necesario), ten en cuenta que esto se escribió con una mentalidad educativa, y hay algunas prácticas de programación muy cuestionables.

In [ ]:
class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

# Renamed from https://github.com/karpathy/ng-video-lecture/blob/52201428ed7b46804849dea0b3ccf0de9df1a5c3/bigram.py#L61
class ToyLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

Todo lo anterior solo sirve para definir una función (enorme), instanciada como

In [ ]:
model = ToyLanguageModel().to(device)

Evalúemos `model` sobre el batch anterior, `xb` (no fue generado con los parámetros que estamos considerando ahora, pero esto no es un problema).

In [ ]:
yb_est = model(xb.to(device))

<font color='red'>TO-DO</font>: ¿Qué devuelve? Explica las dimensiones de cada `Tensor`.

## Bucle de entrenamiento

Una función para estimar la *función de pérdida*, una medida de "lo bien que lo estamos haciendo".

In [ ]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X.to(device), Y.to(device))
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

Código "genérico" para entrenar un modelo. Llegado este punto, esto ya es bastante "comprensible" para ti. En cualquier caso, céntrate simplemente en saber "qué está pasando" a alto nivel.

In [ ]:
model = ToyLanguageModel()

m = model.to(device)

# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb.to(device), yb.to(device))
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

<font color='red'>TO-DO</font>: ¿Está garantizado que has entrenado sobre el dataset *completo* (es decir, que se ha usado cada carácter)?

<font color='red'>TO-DO</font>: Echa un vistazo a las dimensiones de `logits` (de la última iteración en el bucle de entrenamiento), y explícalas. Los `logits` vienen a ser probabilidades antes de ser *normalizadas* para que sumen $1$.

Para utilizar el modelo entrenado para generar texto nuevo, necesitamos configurar un contexto (una especie de punto de partida).

In [ ]:
context = torch.zeros((1, 1), dtype=torch.long, device=device)

<font color='red'>TO-DO</font>: ¿Cuál es el texto asociado con el contexto?

Generemos texto (hasta 2.000 caracteres) usando el contexto anterior

In [ ]:
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))

# Experimentos

- <font color='red'>TO-DO</font>: Entrena el modelo durante menos tiempo (es decir, sobre un número menor de batches, digamos 10), e intenta generar texto. ¿Qué observas? ¿qué diferencia hay entre la *función de pérdida* al final del entrenamiento con la del primer modelo que entrenaste? Entrena el modelo durante más tiempo y responde otra vez a las preguntas.

- <font color='red'>TO-DO</font>: Prueba diferentes tamaños de bloque. ¿Puedes conseguir mejores resultados?

- <font color='red'>TO-DO</font>: Prueba con un dataset diferente (al de Shakespeare) más pequeño. Podrías, por ejemplo, poner la letra de tu canción favorita (un dataset muy pequeño) en la variable `text` de arriba. ¿Qué observas? Compara los valores de la *función de pérdida* para *training* y *validation*.

# Preguntas de ejemplo

## ¿Qué controla el tamaño del *batch* durante el entrenamiento?
- [ ] Cuántas capas tiene el modelo  
- [ ] El número de épocas de entrenamiento
- [ ] Cuántos trozos pequeños de datos se procesan en paralelo antes de actualizar los parámetros del modelo
- [ ] El número máximo de caracteres que se generan

---

## ¿Qué está intentando aprender el modelo durante el entrenamiento?
- [ ] El significado de palabras y frases  
- [ ] Las reglas gramaticales del inglés  
- [ ] El tono emocional de las obras de Shakespeare
- [ ] La probabilidad del siguiente carácter dado los anteriores